In [1]:
!pip install indic-nlp-library
!pip install stopwordsiso

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.0 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import emoji
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from indicnlp.tokenize import indic_tokenize
import stopwordsiso
from torch.utils.data import Dataset, DataLoader
import gensim
from sklearn.metrics import f1_score
import torch.optim as optim
from tqdm import tqdm

nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)
nltk.download("omw-1.4", quiet=True)

True

In [3]:
# i don't know why but this makes the wordnet error disappear

from spacy import load
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')
nlp = load('en_core_web_sm')

! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet # temp fix for lookup error.

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/wordnet2022.zip.


In [4]:
# just loading all the training and testing dataset

dataset_train_english1=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/training/train_en_l1.csv")
dataset_train_english2=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/training/train_en_l2.csv")
dataset_train_english3=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/training/train_en_l3.csv")
dataset_train_english=pd.concat([dataset_train_english1,dataset_train_english2,dataset_train_english3],ignore_index=True)

dataset_train_hindi1=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/training/train_hi_l1.csv")
dataset_train_hindi2=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/training/train_hi_l2.csv")
dataset_train_hindi3=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/training/train_hi_l3.csv")
dataset_train_hindi=pd.concat([dataset_train_hindi1,dataset_train_hindi2,dataset_train_hindi3],ignore_index=True)

dataset_train_tamil1=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/training/train_ta_l1.csv")
dataset_train_tamil2=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/training/train_ta_l2.csv")
dataset_train_tamil3=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/training/train_ta_l3.csv")
dataset_train_tamil=pd.concat([dataset_train_tamil1,dataset_train_tamil2,dataset_train_tamil3],ignore_index=True)

dataset_test_english1=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/testing/test_en_l1.csv")
dataset_test_english2=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/testing/test_en_l2.csv")
dataset_test_english3=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/testing/test_en_l3.csv")
dataset_test_english=pd.concat([dataset_test_english1,dataset_test_english2,dataset_test_english3],ignore_index=True)


dataset_test_hindi1=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/testing/test_hi_l1.csv", encoding="utf-8", on_bad_lines="skip",engine='python')
dataset_test_hindi2=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/testing/test_hi_l2.csv", encoding="utf-8", on_bad_lines="skip",engine='python')
dataset_test_hindi3=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/testing/test_hi_l3.csv", encoding="utf-8", on_bad_lines="skip",engine='python')
dataset_test_hindi=pd.concat([dataset_test_hindi1, dataset_test_hindi2, dataset_test_hindi3], ignore_index=True)

dataset_test_tamil1=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/testing/test_ta_l1.csv", encoding="utf-8", on_bad_lines="skip")
dataset_test_tamil2=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/testing/test_ta_l2.csv", encoding="utf-8", on_bad_lines="skip")
dataset_test_tamil3=pd.read_csv("/kaggle/input/nlp-project-dataset/uli_dataset-main/testing/test_ta_l3.csv", encoding="utf-8", on_bad_lines="skip")
dataset_test_tamil=pd.concat([dataset_test_tamil1,dataset_test_tamil2,dataset_test_tamil3],ignore_index=True)

print(dataset_train_english.columns)

Index(['text', 'key', 'en_a1', 'en_a2', 'en_a3', 'en_a4', 'en_a5', 'en_a6'], dtype='object')


In [5]:
def convert_emojis(text):
    return emoji.demojize(text, delimiters=(" ", " "))  

def remove_mentions(text):
    return re.sub(r"@\w+", "<USER>", text)

def reduce_repetitions(text):
    return re.sub(r'(.)\1+', r'\1\1', text) 

In [6]:
""" if number of annotator  who classify a  tweet as abuse is greater than number of annotator who do not  or numeber of annotator who classify as

    abuse is greater than equal to 2 , then we classify the tweet as abuse ."""  

def preprocess_dataset(dataset,language):
    
    dataset.columns=dataset.columns.str.strip()
    
    dataset=dataset.drop(columns=["key"])
    dataset.dropna(subset=["text"], inplace=True)
    abuse=[]

    # classifying the tweet as abuse or not
    
    for index,record in dataset.iterrows():
        count_0=0
        count_1=0
        for keys in record.keys():
            if (keys!='text'):
                if (record[keys]==1):count_1+=1
                if (record[keys]==0):count_0+=1
        
        if (count_1>=count_0  or count_1>1 ):abuse.append(1)
        else:abuse.append(0)
    # keeping only relevant columns in the dataset
    
    dataset['abuse']=abuse
    dataset=dataset[['text','abuse']]

    # preprocessing the text

    dataset['text']=dataset['text'].apply(remove_mentions) # remove user's name with a tag <USER>
    dataset['text']= dataset['text'].apply(convert_emojis) # convert emojies into text description
    dataset["text"]=dataset["text"].str.replace(r"http\S+|www\S+", "<URL>", regex=True) # removes any urls with tag <URL>
    dataset["text"]=dataset["text"].str.replace(r"[^A-Za-z0-9\s]", "", regex=True).str.strip() #removes any special character and empty spaces

    if (language=='en'):
        dataset["text"] = dataset["text"].apply(nltk.word_tokenize)
        stop_words = set(stopwords.words("english"))
        dataset["text"] = dataset["text"].apply(lambda x: [word for word in x if word.lower() not in stop_words])
        
        lemmatizer = WordNetLemmatizer()
        dataset["text"] = dataset["text"].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
        dataset["text"] = dataset["text"].apply(lambda x: " ".join(x))
        dataset['text'] = dataset['text'].apply(reduce_repetitions)
    else:
        
        dataset["text"] = dataset["text"].apply(lambda x: indic_tokenize.trivial_tokenize(x, lang=language))
        language_stopwords=stopwordsiso.stopwords(language)
        dataset["text"] = dataset["text"].apply(lambda x: [word for word in x if word not in language_stopwords])
        dataset["text"] = dataset["text"].apply(lambda x: " ".join(x))

    return dataset
        
    


    

In [7]:

dataset_train_english=preprocess_dataset(dataset_train_english,language='en')
dataset_test_english=preprocess_dataset(dataset_test_english,language='en')
print("english dataset preprocessed")
dataset_train_hindi=preprocess_dataset(dataset_train_hindi,language='hi')
dataset_test_hindi=preprocess_dataset(dataset_test_hindi,language='hi')
print("hindi dataset preprocessed")
dataset_train_tamil=preprocess_dataset(dataset_train_tamil,language='ta')
dataset_test_tamil=preprocess_dataset(dataset_test_tamil,language='ta')
print('tamil dataset preprocessed')

english dataset preprocessed
hindi dataset preprocessed
tamil dataset preprocessed


In [8]:
from transformers import BertTokenizer, BertForSequenceClassification

MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [9]:
class AbuseDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        encoded = self.tokenizer(
            self.texts[idx], 
            padding="max_length", 
            truncation=True, 
            max_length=self.max_length, 
            return_tensors="pt"
        )
        input_ids = encoded["input_ids"].squeeze()
        attention_mask = encoded["attention_mask"].squeeze()
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return input_ids, attention_mask, label



In [28]:
from tqdm import tqdm

def train(training_dataset, testing_dataset, Model_name):
    model = BertForSequenceClassification.from_pretrained(Model_name, num_labels=2)  
    tokenizer = BertTokenizer.from_pretrained(Model_name)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    train_dataset = AbuseDataset(training_dataset["text"].tolist(), training_dataset["abuse"].tolist(), tokenizer)
    test_dataset = AbuseDataset(testing_dataset["text"].tolist(), testing_dataset["abuse"].tolist(), tokenizer)
    
    BATCH_SIZE = 8
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=2e-5)
    
    EPOCHS = 0
    model.train()
    
    for epoch in range(EPOCHS):
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=True)  
        
        for batch in progress_bar:  
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item()) 
        
        print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss:.4f}")

    # Evaluation
    model.eval()
    y_preds = []
    y_true = []
    
    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc="Evaluating", leave=True)  

        for batch in progress_bar:  
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask).logits
            preds = torch.argmax(outputs, dim=1)
            y_preds.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

        progress_bar.close() 
    
    macro_f1 = f1_score(y_true, y_preds, average="macro")
    print(f"Macro F1-score: {macro_f1:.4f}")


In [29]:
import gc
torch.cuda.empty_cache()  
gc.collect()  
print("*"*20)
print('Result for English Language')
train(dataset_train_english,dataset_test_english,MODEL_NAME)
print()

print("*"*20)
print('Result for Hindi Language')
train(dataset_train_hindi,dataset_test_hindi,MODEL_NAME)
print()

print("*"*20)
print('Result for Tamil Language')
train(dataset_train_tamil,dataset_test_tamil,MODEL_NAME)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


********************
Result for English Language


Evaluating: 100%|██████████| 416/416 [00:27<00:00, 14.87it/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Macro F1-score: 0.5083

********************
Result for Hindi Language


Evaluating: 100%|██████████| 569/569 [00:38<00:00, 14.91it/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Macro F1-score: 0.5063

********************
Result for Tamil Language


Evaluating: 100%|██████████| 426/426 [00:27<00:00, 15.44it/s]

Macro F1-score: 0.3669
